Webscraper to get race results and add them to a postgresql database

In [41]:
import asyncio
from playwright.async_api import async_playwright
import json

In [4]:
async def main():
    browsers = ['chromium', 'firefox', 'webkit']
    async with async_playwright() as p:
        for browser_type in browsers:
            browser = await p[browser_type].launch()
            page = await browser.new_page()
            await page.goto('https://amazon.com')
            await page.screenshot(path=f'py_{browser_type}.png', full_page=True)
            await page.wait_for_timeout(1000)
            await browser.close()
            
await main()

In [52]:
all_rows_data = []
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False) # headless=False lets you watch
    page = await browser.new_page()
        
    # Go to the page where the button will appear
    await page.goto("https://track.rtrt.me/e/ATC-GRANDPRIX-1MILE-2025#/leaderboard/overall-sex-by-net-time-1mile-_ALL/FINISH")
        
    print("Page loaded. Looking for the 'Spectator' button...")
        
    # Use a locator to find the button. Playwright will automatically wait
    # for it to appear before proceeding.
        
        
    #await page.pause()
    # Click the button
    page_number = 1
    await page.locator("iframe[name=\"rtframe\"]").content_frame.get_by_role("button", name="I'm a Spectator").click()
    frame_locator = page.frame_locator("iframe[name=\"rtframe\"]")
    next = frame_locator.locator("#topleaderheight").get_by_role("button", name="next")
    print("Successfully clicked the 'Spectator' button.")
        
    # Now you can proceed to wait for your results and scrape them
    # For example, wait for a results container to be visible:
    while True:
        print(page_number)
        
        await frame_locator.locator("tr.leaderrow").first.wait_for()
        rows = frame_locator.locator("tr.leaderrow")
            
        rows_count = await rows.count()
        print(f"Found {rows_count} rows.")
        
        for row in await rows.all():
            name = await row.locator(".lbname").text_content()
            time = await row.locator(".lbtime").text_content()
            row_data = {"name": name.strip(), "time": time.strip()}
            all_rows_data.append(row_data)
        print("\n--- All Table Data ---")
        print(all_rows_data)
        
        if await next.is_visible():
            await next.click()
            page_number += 1
            await page.wait_for_timeout(1000)
            await page.wait_for_load_state("networkidle")
        else:
            print('last page')
            break
    await page.screenshot(path=f'next.png', full_page=True)
    # Add a small delay to see the result
    await page.wait_for_timeout(5000)
        
    await browser.close()
print(json.dumps(all_rows_data, indent=2))


Page loaded. Looking for the 'Spectator' button...
Successfully clicked the 'Spectator' button.
1
Found 50 rows.

--- All Table Data ---
[{'name': 'Everett Smulders\n                        E. Smulders\n                    Bib 2759   M, 26   Atlanta, GA', 'time': '00:04:24\n                    \n\n                       \n                    ---'}, {'name': 'Will Ponder\n                        Will Ponder\n                    Bib 2753   M, 24   Atlanta, GA', 'time': '00:04:30\n                    \n\n                       \n                    +00:05.37'}, {'name': 'Nick French\n                        Nick French\n                    Bib 2595   M, 31   Atlanta, GA', 'time': '00:04:32\n                    \n\n                       \n                    +00:07.36'}, {'name': 'Frank Triolo\n                        Frank Triolo\n                    Bib 2685   M, 24   Atlanta, GA', 'time': '00:04:34\n                    \n\n                       \n                    +00:09.18'}, {'nam

In [53]:
print(len(all_rows_data))

240


Testing peachtree just out of curiousity

In [54]:
url = "https://track.rtrt.me/e/ATC-PEACHTREE-2024#/leaderboard/overall-by-net-time-all-participants-10k-_ALL/FINISH"

all_rows_data = []
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False) # headless=False lets you watch
    page = await browser.new_page()
        
    # Go to the page where the button will appear
    await page.goto(url)
        
    print("Page loaded. Looking for the 'Spectator' button...")
        
    # Use a locator to find the button. Playwright will automatically wait
    # for it to appear before proceeding.
        
        
    #await page.pause()
    # Click the button
    page_number = 1
    await page.locator("iframe[name=\"rtframe\"]").content_frame.get_by_role("button", name="I'm a Spectator").click()
    frame_locator = page.frame_locator("iframe[name=\"rtframe\"]")
    next = frame_locator.locator("#topleaderheight").get_by_role("button", name="next")
    print("Successfully clicked the 'Spectator' button.")
        
    # Now you can proceed to wait for your results and scrape them
    # For example, wait for a results container to be visible:
    while page_number < 10:
        print(page_number)
        
        await frame_locator.locator("tr.leaderrow").first.wait_for()
        rows = frame_locator.locator("tr.leaderrow")
            
        rows_count = await rows.count()
        print(f"Found {rows_count} rows.")
        
        for row in await rows.all():
            name = await row.locator(".lbname").text_content()
            time = await row.locator(".lbtime").text_content()
            row_data = {"name": name.strip(), "time": time.strip()}
            all_rows_data.append(row_data)
        print("\n--- All Table Data ---")
        print(all_rows_data)
        
        if await next.is_visible():
            await next.click()
            page_number += 1
            await page.wait_for_timeout(1000)
            await page.wait_for_load_state("networkidle")
        else:
            print('last page')
            break
    await page.screenshot(path=f'next.png', full_page=True)
    # Add a small delay to see the result
    await page.wait_for_timeout(5000)
        
    await browser.close()
print(json.dumps(all_rows_data, indent=2))


Page loaded. Looking for the 'Spectator' button...
Successfully clicked the 'Spectator' button.
1
Found 15 rows.

--- All Table Data ---
[{'name': 'Sabastian Sawe\n                        S. Sawe\n                    Bib 29  M-29  ,  KEN', 'time': '00:28:03\n                    \n\n                       \n                    ---'}, {'name': 'Daniel Ebenyo\n                        D. Ebenyo\n                    Bib 37  M-28  ,  KEN', 'time': '00:28:05\n                    \n\n                       \n                    +00:01.07'}, {'name': 'Robert Kiprop Koech\n                        R. K. Koech\n                    Bib 18  M-27  ,  KEN', 'time': '00:28:18\n                    \n\n                       \n                    +00:14.36'}, {'name': 'Hillary Kipkoech\n                        H. Kipkoech\n                    Bib 17  M-25  ,  KEN', 'time': '00:28:20\n                    \n\n                       \n                    +00:16.79'}, {'name': 'Boniface Kibiwott\n           

Update: it works